# Manual Brand Report Refresh - DCM

In [1]:
!git checkout dev
from brand_reporting_dags.dfa_reporting_dag_tasks import dfa_report_extract,dfa_report_load,clean_up

import json
import os
from datetime import datetime
import time
import warnings
warnings.filterwarnings("ignore", "Your application has authenticated using end user credentials")

M	bonito_Q3_refresh.ipynb
M	brand_reporting_dags/dfa_reporting_dag_tasks.py
D	dag playground.ipynb
M	rtf_utils/__pycache__/dfa_utils.cpython-36.pyc
M	rtf_utils/__pycache__/gcp_utils.cpython-36.pyc
M	rtf_utils/dfa_utils.py
Already on 'dev'
Your branch is up to date with 'origin/dev'.


In [2]:
from rtf_utils.dfa_utils import CampaignManagerReport,get_dfa_report,clean_dcm_file
from rtf_utils.gcp_utils import BigQuery,CloudStorage
from essence.analytics.platform import securedcredentials as secure_creds

## DCM Reports

In [25]:
with open('dags/brand_report_task_queue.json') as json_file:
    config_json = json.load(json_file)
    campaigns = config_json['campaigns']

Task = (report_id, bq_dest (proj:dataset.table), schema)

In [3]:
tasks = campaigns[0]['dcmTasks']

In [5]:
start_date = '2019-09-01'
end_date = '2019-09-10'
gcs_files = [dfa_report_extract(task['reportId'],start_date = start_date,end_date = end_date) for task in tasks]

Get Report Meta
File is already available. Start download
File downloaded to Bonito_Classification.csv
Cleaning File
Auth GCS
Upload File
Clean Up Local
Get Report Meta
File doesn't exist, run report
The Download Part
Report Processing, Wait 10. Try 0
Report Processing, Wait 10. Try 1
Print Report Available!
File downloaded to Bonito_Video_Player_Size.csv
Cleaning File
Auth GCS
Upload File
Clean Up Local


In [4]:
gcs_files = [('rtf_staging', 'brand_reporting/2019-09-10_Bonito_Classification.csv'),
 ('rtf_staging', 'brand_reporting/2019-09-10_Bonito_Video_Player_Size.csv')]

In [5]:
service_account_email='131786951123-compute@developer.gserviceaccount.com'
credentialsFromVault=secure_creds.getCredentialsFromEssenceVault(service_account_email)
bq = BigQuery(credentialsFromVault)

for file in gcs_files:
    
    bucket, filename = file
    file_uri = "gs://" + bucket + "/" + filename
    dataset_id = 'rtf_brand_reporting'
    
    if 'Classification' in filename:
        dest_table = 'DCM_contentVerification' 
    else:
        dest_table = 'DCM_playerSize'
    print("Loading {} into {}".format(filename,dest_table))
    bq.load_from_gcs(dataset_id,file_uri,dest_table,mode='Append')

Loading brand_reporting/2019-09-10_Bonito_Classification.csv into DCM_contentVerification
Loading brand_reporting/2019-09-10_Bonito_Video_Player_Size.csv into DCM_playerSize


In [12]:
new_report = {'name': 'geo_report', 'profileId': 5096586, 'reportId': 633170439}
config_json['campaigns'][0]['dcmTasks'].append(new_report)

In [13]:
config_json['campaigns'][0]['dcmTasks']

[{'name': 'verification_report', 'profileId': 5096586, 'reportId': 631165400},
 {'name': 'video_player_size', 'profileId': 5096586, 'reportId': 632709822},
 {'name': 'geo_report', 'profileId': 5096586, 'reportId': 633170439}]

In [20]:
config_json['campaigns'].extend(new_campaigns)

In [24]:
with open('dags/brand_report_task_queue.json','w') as json_file:
    json.dump(config_json,json_file)
    

In [58]:
dcm_tasks = []
for campaign in config_json['campaigns']:
     dcm_tasks.extend(campaign['dcmTasks'])

In [47]:
print(sorted(dcm_tasks, key = lambda i: i['name']))

[{'name': 'geo_report', 'profileId': 5096586, 'reportId': 633170439}, {'name': 'geo_report', 'profileId': 5096586, 'reportId': ''}, {'name': 'geo_report', 'profileId': 5096586, 'reportId': ''}, {'name': 'geo_report', 'profileId': 5096586, 'reportId': ''}, {'name': 'verification_report', 'reportId': 631165400, 'profileId': 5096586}, {'name': 'verification_report', 'profileId': 5096586, 'reportId': ''}, {'name': 'verification_report', 'profileId': 5096586, 'reportId': ''}, {'name': 'verification_report', 'profileId': 5096586, 'reportId': ''}, {'name': 'video_player_size', 'reportId': 632709822, 'profileId': 5096586}, {'name': 'video_player_size', 'profileId': 5096586, 'reportId': ''}, {'name': 'video_player_size', 'profileId': 5096586, 'reportId': ''}, {'name': 'video_player_size', 'profileId': 5096586, 'reportId': ''}]


In [61]:
def task_orchestrator(campaign_list,):
    dcm_tasks = []
    
    for campaign in config_json['campaigns']:
         dcm_tasks.extend(campaign['dcmTasks'])


    prev_task_name = ""

    tasks_sorted = sorted(dcm_tasks, key = lambda i: i['name'])

    while tasks_sorted:
        task = tasks_sorted.pop()
        task_name = task['name']
        if task_name != prev_task_name:
            print("Starting {}".format(task_name))
        prev_task_name = task_name
        print(task['reportId'])


Starting video_player_size



632709822
Starting verification_report



631165400
Starting geo_report



633170439
